In [1]:
import pandas as pd
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [10]:
#Para inicio do exercicio vou utilizar a mesma base de dados do vídeo para conseguir um critério de comparação
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data',header=None,names=['buying','maint','doors','persons','lug_boot','safety','evaluation'])
df.describe()

,buying,maint,doors,persons,lug_boot,safety,evaluation
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,high,high,5more,4,big,high,unacc
freq,432,432,432,576,576,576,1210


In [11]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [12]:
df.shape

(1728, 7)

In [5]:
#Dando um count para cada valor de buying e maint com resultado em evaluation para avaliar os números
df.value_counts(subset=['buying','maint','evaluation'],sort=False)

buying  maint  evaluation
high    high   acc            36
               unacc          72
        low    acc            36
               unacc          72
        med    acc            36
               unacc          72
        vhigh  unacc         108
low     high   acc            33
               unacc          62
               vgood          13
        low    acc            10
               good           23
               unacc          62
               vgood          13
        med    acc            10
               good           23
               unacc          62
               vgood          13
        vhigh  acc            36
               unacc          72
med     high   acc            36
               unacc          72
        low    acc            10
               good           23
               unacc          62
               vgood          13
        med    acc            33
               unacc          62
               vgood          13
        vhigh  ac

In [6]:
#primeira versão da árvore, apenas para atributos categóricos
class Arvore(BaseEstimator, ClassifierMixin):
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.tree = self.eval_depth(X,y)
        return self.tree
       
    def eval_depth(self, X, y):
        list_tree=[]
        if(len(X.columns)==0):
            return [(y.name,y.value_counts(sort=False,normalize=True).idxmax())]

        d={}
        for column in X.columns:
            r=0
            for value in X[column].unique():
                result = y[X.loc[X[column]==value].index].unique()
                r+=len(result)
            d[column]=r
        selection = max(d, key=d.get)

        for value in X[selection].unique():
            subarvore = Arvore()
            list_tree.append([(selection,value),subarvore.eval_depth(X.loc[X[selection]==value].drop(labels=[selection],axis=1),y[X.loc[X[selection]==value].index])])

        return list_tree
    
    
    def predict(self,X):
        self.y_pred=[]
        for i in X.index:
            self.y_pred.append(self.check_X(X.loc[i],self.tree,()))
        return self.y_pred
        
        
    def check_X(self,x,list_tree,actual_tuple):
        if (len(x)==0): return list_tree[0][1]

        for i in range(len(list_tree)):
            if(x[list_tree[i][0][0]]==list_tree[i][0][1]): 
                return self.check_X(x.drop(labels=[list_tree[i][0][0]]),list_tree[i][1],list_tree[i][0])
            
        return self.define_X(actual_tuple)
    
    
    def define_X(self, actual_tuple):
        return self.y[self.X.loc[self.X[actual_tuple[0]]==actual_tuple[1]].index].value_counts(sort=False,normalize=False).idxmax()

In [7]:
#teste para uma parcela do dataset, pode ser visualizado na árvore
df_resumido = df.loc[((df.maint == 'high') | (df.maint == 'vhigh')) & ((df.doors == '2') | (df.doors == '4'))].drop(labels=['buying','persons','lug_boot','safety'],axis=1).copy()

arvore = Arvore()
X = df_resumido.drop(labels=['evaluation'],axis=1)
y = df_resumido['evaluation']
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
accuracy_score(y, y_pred)

0.7916666666666666

In [8]:
#teste para todo o dataset com ressubstituição
arvore = Arvore()
X = df.drop(labels=['evaluation'],axis=1)
y = df['evaluation']
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
accuracy_score(y, y_pred)

1.0

In [9]:
#teste com split de treino/teste
X = df.drop(labels=['evaluation'],axis=1)
y = df['evaluation']

arvore = Arvore()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

tree = arvore.fit(X_train,y_train)
y_pred = arvore.predict(X_test)

accuracy_score(y_test, y_pred)

0.6952714535901926

In [10]:
modelo = Arvore()
scores = cross_val_score(modelo, X, y, cv=4)
print("Scores:",scores)
print("Média dos scores:",scores.mean())

Scores: [0.70138889 0.70138889 0.69907407 0.54166667]
Média dos scores: 0.6608796296296295


<hr style="height:2px;border-width:0;color:black;background-color:black">
Árvore com atributos contínuos

In [2]:
#fazendo load no dataset da Iris
from sklearn.datasets import load_iris
iris = load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df.loc[:,'target'] = iris.target

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
df.shape

(150, 5)

In [2]:
#segunda versão da árvore, agora considerando atributos contínuos e categóricos
class Arvore(BaseEstimator, ClassifierMixin):
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.tree = self.eval_depth(X,y)
        return self.tree
       
    def eval_depth(self, X, y):
        #essa forma que escolhi de selecionar as características foi muito ruim, porque primeiro ele faz um select das categóricas, e depois das contínuas. Se fosse um dataset somente dessas características, como nos testes, passa ok, mas para datasets que tem conjuntos misturados fica enviezado.
        list_tree=[]
        if(len(X.columns)==0):
            #print(y)
            return [(y.name,y.value_counts(sort=False,normalize=True).idxmax())]
        
                   
        #seleção da característica dos tipos categóritcos: baseada no número de valores baseados no target, é selecionada a característica com maior número de valores do target x número de valores da característica. Pensei nisso para "arborizar" a árvore
        d={}
        for column in X.select_dtypes(include=['object']).columns:
            r=0
            for value in X[column].unique():
                result = y[X.loc[X[column]==value].index].unique()
                r+=len(result)
            d[column]=r
            
        if(d!={}):
            selection_objects = max(d, key=d.get)
            for value in X[selection_objects].unique():
                #print("categorico",value)
                subarvore = Arvore()
                list_tree.append([(selection_objects,value),subarvore.eval_depth(X.loc[X[selection_objects]==value].drop(labels=[selection_objects],axis=1),y[X.loc[X[selection_objects]==value].index])])
                          
        #seleção da característica contínua: escolhida a característica que tem menor diferença entre a mediana e a média, pois assim a característica está "bem distribuída". Para montagem da árvore nesse caso é feita uma conta em que o valor atual da instância tem que estar no intervalo da mediana +ou- o desvio padrão 
        d={}
        for column in X.select_dtypes(include=['int32','float32','int64','float64']).columns:
            d[column]=abs(X[column].median()-X[column].mean())
            
        if(d!={}):
            selection_continuo = min(d, key=d.get)
            for value in y.unique():
                #print("continuo",value)
                median = pd.concat([X.loc[:,selection_continuo],y],axis=1).groupby(by=[y.name]).median().loc[value]
                std = pd.concat([X.loc[:,selection_continuo],y],axis=1).groupby(by=[y.name]).std().loc[value]
                index = X.loc[(X[selection_continuo] >= float(median-std)) & (X[selection_continuo] < float(median+std))].index
                subarvore = Arvore()
                if(not index.empty):
                    list_tree.append([(selection_continuo,(median,std)),subarvore.eval_depth(X.loc[index].drop(labels=[selection_continuo],axis=1),y[index])])
            

        return list_tree
    
    
    def predict(self,X):
        #predict passa por todo o vetor fazendo uma recursão na árvore gerada no fit
        self.y_pred=[]
        for i in X.index:
            self.y_pred.append(self.check_X(X.loc[i],self.tree,()))
        return self.y_pred
        
        
    def check_X(self,x,list_tree,actual_tuple):
        #o check_X faz uma validação. Se chegou na folha da árvore, retorna o segundo item da tupla, que é a classificação. Se a instância não obedece a árvore, foi definida uma função define_X para chutar um valor.
        if (len(x)==0): return list_tree[0][1]

        for i in range(len(list_tree)):
            if(isinstance(x[list_tree[i][0][0]],str)):

                if(x[list_tree[i][0][0]]==list_tree[i][0][1]): 
                    return self.check_X(x.drop(labels=[list_tree[i][0][0]]),list_tree[i][1],list_tree[i][0])
            else:
                if((x[list_tree[i][0][0]] >= float(list_tree[i][0][1][0] - list_tree[i][0][1][1])) & (x[list_tree[i][0][0]] < float(list_tree[i][0][1][0] + list_tree[i][0][1][1]))): 
                    return self.check_X(x.drop(labels=[list_tree[i][0][0]]),list_tree[i][1],list_tree[i][0])
        return self.define_X(actual_tuple)
    
    
    def define_X(self, actual_tuple):
        #nessa função é feita uma avaliação não muito precisa da característica. 
        #Se for categórica, é filtrado pela característica e pelo valor categórico que está sendo pesquisado, e pego a contagem de maior valor do target.
        #Se for contínua é avaliado em qual target essa característica está com menos desvio da mediana, e retornada essa classificação.
        #Se por um azar a primeira ramificação da árvore não encontrar nenhum valor nos para seguir a classificação, vem uma tupla vazia, aí é retornado a contagem da maior classificação do target
        if(len(actual_tuple) == 0): return Counter(self.y).most_common(1)[0][0]
        if(isinstance(actual_tuple[1],tuple)):
            return abs(actual_tuple[1][0] - pd.concat([self.X.loc[:,actual_tuple[0]],self.y],axis=1).groupby(by=[self.y.name]).median()).idxmin()[0]
        else: 
            return self.y[self.X.loc[self.X[actual_tuple[0]]==actual_tuple[1]].index].value_counts(sort=False,normalize=False).idxmax()

In [16]:
#testando para todo o dataset com ressubstituição
arvore = Arvore()
X = df.drop(labels=['target'],axis=1)
y = df['target']
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
accuracy_score(y, y_pred)

0.7933333333333333

In [17]:
#testando com split
arvore = Arvore()
X = df.drop(labels=['target'],axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

tree = arvore.fit(X_train,y_train)
y_pred = arvore.predict(X_test)

accuracy_score(y_test, y_pred)

0.64

In [18]:
#testando com cross_validate
modelo = Arvore()
scores = cross_val_score(modelo, X, y, cv=4)
print("Scores:",scores)
print("Média dos scores:",scores.mean())

Scores: [0.68421053 0.65789474 0.78378378 0.89189189]
Média dos scores: 0.7544452347083925


<hr style="height:2px;border-width:0;color:black;background-color:black">
Testando com o dataset de estrelas, utilizado nas tarefas anteriores

In [6]:
dataset = "https://raw.githubusercontent.com/Leovsimoes/ReconhecimentoDePadroes/main/6%20class%20csv.csv"
df = pd.read_csv(dataset)
df.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [7]:
df.shape

(240, 7)

In [9]:
#testando com ressubstituição, split e cross_validate
arvore = Arvore()
X = df.drop(labels=['Star type'],axis=1).dropna(axis=1)
y = df['Star type']
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
print("Ressubstituição:",accuracy_score(y, y_pred))

arvore = Arvore()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

tree = arvore.fit(X_train,y_train)
y_pred = arvore.predict(X_test)
print("Split:",accuracy_score(y_test, y_pred))

#testando com cross_validate
modelo = Arvore()
scores = cross_val_score(modelo, X, y, cv=4)
print("Média dos scores do cross_validate:",scores.mean())

Ressubstituição: 0.7583333333333333
Split: 0.725
Média dos scores do cross_validate: 0.6041666666666667


<hr style="height:2px;border-width:0;color:black;background-color:black">
Testando com o dataset de vinhos, que iria ser utilizado na tarefa de classificação

In [8]:
dataset = "https://raw.githubusercontent.com/Leovsimoes/ReconhecimentoDePadroes/main/winequalityN.csv"
df = pd.read_csv(dataset)
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
df.shape

(6497, 13)

In [11]:
#testando com ressubstituição, split e cross_validate

#tive que adicionar o dropna para colunas aqui, pois não deixava o algoritmo rodar.
X = df.drop(labels=['quality'],axis=1).dropna(axis=1)
y = df['quality']

arvore = Arvore()
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
print("Ressubstituição:",accuracy_score(y, y_pred))

arvore = Arvore()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

tree = arvore.fit(X_train,y_train)
y_pred = arvore.predict(X_test)
print("Split:",accuracy_score(y_test, y_pred))

#testando com cross_validate
modelo = Arvore()
scores = cross_val_score(modelo, X, y, cv=4)
print("Média dos scores do cross_validate:",scores.mean())

Ressubstituição: 0.42527320301677696
Split: 0.42424242424242425
Média dos scores do cross_validate: 0.37863205759757484


<hr style="height:2px;border-width:0;color:black;background-color:black">
Testando para o terceiro dataset, retirado do Kaggle, que diz respeito a aprovação ou não em um campus de universidade americana 
<br/>
https://www.kaggle.com/benroshan/factors-affecting-campus-placement?select=Placement_Data_Full_Class.csv

In [13]:
dataset = "https://raw.githubusercontent.com/Leovsimoes/ReconhecimentoDePadroes/main/Placement_Data_Full_Class.csv"
df = pd.read_csv(dataset)
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [18]:
df.shape

(215, 15)

In [19]:
#testando com ressubstituição, split e cross_validate

X = df.drop(labels=['sl_no','status'],axis=1).dropna(axis=1)
y = df['status']

arvore = Arvore()
tree = arvore.fit(X,y)
y_pred = arvore.predict(X)
print("Ressubstituição:",accuracy_score(y, y_pred))

arvore = Arvore()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

tree = arvore.fit(X_train,y_train)
y_pred = arvore.predict(X_test)
print("Split:",accuracy_score(y_test, y_pred))

#testando com cross_validate
modelo = Arvore()
scores = cross_val_score(modelo, X, y, cv=4)
print("Média dos scores do cross_validate:",scores.mean())

Ressubstituição: 0.7674418604651163
Split: 0.5774647887323944
Média dos scores do cross_validate: 0.6466631726065688
